In [1]:
import numpy as np
import gemmi
from matplotlib import pyplot as plt

In [ ]:
# the "FAD frame" is:
#  +x from C10 to C9A
#  +y from N5 to N10
#  +z completing the right hand rule

TDM_IN_FAD_FRAME = np.array([])

In [24]:
def tdm_in_crystal_frame(trj, chain=0):
    return


def rotation_matrix_from_A(A):
    """
    For an orthorhombic system, we can write the A matrix as:

      A = R S I

    Where:
      -- I is the identity
      -- S is a (diagonal) scaling matrix that sets the length of each a/b/c/ axis
      -- R is a rotation matrix

    So, we'll simply write:
    
      R = A S^{-1}
    """

    if not A.shape == (3,3):
        raise ValueError('A not 3x3', A.shape)

    S = np.diag([70.200,  117.760,  170.470]) * 0.1  # cell lengths in nm (CrystFEL uses nm)
    R = np.dot(A, np.linalg.inv(S))

    # rotation matrices should have det(R) = 1
    det = np.linalg.det(R)
    assert np.abs(det - 1.0) < 0.01, det

    return R


def expand_p212121(vector):

    sg = gemmi.SpaceGroup('P212121')
    ops = sg.operations()

    set_of_vectors = []
    for op in ops:
        vt = op.apply_to_xyz(vector)
        set_of_vectors.append(vt)

    return np.array(set_of_vectors)

### TDM

Logic. Given TDM $\mu$ in the FAD frame of reference, we need to compute the set of TDMs in the experimental frame. To do this, we:

1. for each chain: compute the rotation matrices to go from the crystal frame to the frame of FAD (2 rotation matrices)
2. compute the TDM for each chain in the crystal frame (2 vectors)
3. symmetry-expand these vectors (4 + 4 vectors)
4. rotate these vectors using the A-matrix into the experimental frame

Then we can simply compute the component of the TDM that is in the (circular) polarization plane as:

$$
\sum_i 1 - (\vec{\mu}_i \cdot \vec{z})^2
$$

NB! The A-matices were computed such that (a,b,c) formed the ROWS of the matrix, but here we transpose them such that they form the COLUMNS, which is more standard.

In [18]:
a_matrix_file = '/Users/tjlane/Desktop/PL-workshop/signals_chain_A_vs_B/orient/crystal_Amatrices_100us_dark.npy'
#a_matrix_file = '/Users/tjlane/Desktop/PL-workshop/signals_chain_A_vs_B/orient/crystal_Amatrices_TD_all.npy'
#a_matrix_file = '/Users/tjlane/Desktop/PL-workshop/signals_chain_A_vs_B/orient/crystal_Amatrices_100us_l+d.npy'
 
A_matrices = np.transpose(np.load(a_matrix_file), axes=(0,2,1))
print(A_matrices.shape)

(115492, 3, 3)


In [25]:
rotation_matrix_from_A(A_matrices[0])

array([[-0.05561658,  0.23822073, -0.96976568],
       [-0.59976917, -0.78317727, -0.15801732],
       [-0.79152978,  0.5785835 ,  0.17766725]])